In [3]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import glm_model as gm
from glm_utils import *
import tensorflow as tf

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
#finally get rid of the ugly ass allen code. We import the pickled design 
#matrix stimulus, which is PCA projection of the images, and the downsampled
#traces. 

import pickle

pkl_file = open('./boc/formatted/LMcelldata')
data = pickle.load(pkl_file)
pkl_file.close()

pkl_file = open('./boc/formatted/LMstimdata')
stim = pickle.load(pkl_file)
pkl_file.close()

In [5]:
#get the number of features, and the number of training examples. 
l, n =  stim.shape

In [7]:
from statsmodels.tsa.tsatools import lagmat
from sklearn.cross_validation import train_test_split
from scipy.stats import linregress
from itertools import product
import pyprind

import sys

n_pcs = range(5, 110, 5)
non_lin = [1]
non_lin_dict = {1: "sigmoid", 2: "relu"}
alpha = .1

#empty dictionary
best_model = {k: (-np.inf, None) for k in data.keys()}

#make a dictionary of dictionaries to store the results
scores_dict = {k: {} for k in data.keys()}

bar = pyprind.ProgBar(len(n_pcs)*len(non_lin))

for (l, n) in product(non_lin, n_pcs):
    print "testing non-linearity: ", non_lin_dict[l], "... testing ", n, " pc's..."
    sys.stdout.flush()
    
    #lagged_stim = lagmat(stim[:, 0:n], maxlag = nlag, trim = None)[1:-1]         
    bar.update()
    
    for c in data.keys():
        X_train, X_test, y_train, y_test = train_test_split(stim[:, 0:n], np.array(data[c]))    
        
        y_train[y_train < 0 ], y_test[y_test < 0] = 0, 0
        
        weight_init = np.linalg.pinv(X_train).dot(y_train)[:, np.newaxis]

        if l == 1:
            nonlin = tf.nn.sigmoid
        if l == 2:
            nonlin = tf.nn.relu
        
        model = gm.exponential_GLM(weight_init, non_lin = nonlin, verbose = False, lr = 1e-2, alpha = 0)
        cv, loss = model.fit(X_train, y_train[:, np.newaxis], X_test, y_test[:, np.newaxis], max_iters = 100) 
    
    
        _, _, r2_test,_, _ = linregress(np.squeeze(model.predict_trace(X_test)), np.squeeze(y_test))
        _, _, r2_train,_, _ = linregress(np.squeeze(model.predict_trace(X_train)), np.squeeze(y_train))

        scores_dict[c][(l, n)] = (r2_test, r2_train) 

        if r2_test > best_model[c][0]:
            best_model[c] = (r2_test, (l, n))
            print "new best model for cell ", c, ". r2 test: ", r2_test


0%                 100%
[                     ]

testing non-linearity:  sigmoid ... testing  5  pc's...
new best model for cell  517442753 . r2 test:  0.0184834481583
new best model for cell  517444738 . r2 test:  -0.00885479120134
new best model for cell  517443331 . r2 test:  -0.0016966730845
new best model for cell  517442436 . r2 test:  0.0333721905176
new best model for cell  517445575 . r2 test:  0.014990950839
new best model for cell  517444426 . r2 test:  0.00109692855011
new best model for cell  517442471 . r2 test:  -0.0149227473391
new best model for cell  517443181 . r2 test:  -0.0273909255247
new best model for cell  517443087 . r2 test:  -0.0419571644712
new best model for cell  517446833 . r2 test:  0.0268076455904
new best model for cell  517443827 . r2 test:  0.0223687209709
new best model for cell  517443924 . r2 test:  -0.0473309618569
new best model for cell  517443957 . r2 test:  0.0139151027395
new best model for cell  517442722 . r2 test:  0.00475460960377
new best model for cell  517446488 . r2 test:  0.00614

[#                    ] | ETA: 00:10:55

new best model for cell  517443331 . r2 test:  -0.00022275943762
new best model for cell  517444426 . r2 test:  0.04199177105
new best model for cell  517442471 . r2 test:  0.0196384523523
new best model for cell  517443181 . r2 test:  0.0234119112442
new best model for cell  517443087 . r2 test:  0.0256967353733
new best model for cell  517443924 . r2 test:  -0.00364552532513
new best model for cell  517442722 . r2 test:  0.0174814886804
new best model for cell  517442675 . r2 test:  0.000773795710824
new best model for cell  517446554 . r2 test:  -0.0122900691563
new best model for cell  517444175 . r2 test:  0.0175788479285
new best model for cell  517447069 . r2 test:  0.0323632115919
testing non-linearity:  sigmoid ... testing  15  pc's...


[###                  ] | ETA: 00:20:57

...

[###################  ] | ETA: 00:14:24

testing non-linearity:  sigmoid ... testing  100  pc's...


[#################### ] | ETA: 00:07:41

new best model for cell  517442471 . r2 test:  0.046315785297
testing non-linearity:  sigmoid ... testing  105  pc's...


[#####################] | ETA: 00:00:00
Total time elapsed: 02:52:05


In [8]:
pkl_file = open('./boc/formatted/LNE_sig_non_neg', 'wb')
pickle.dump((best_model, scores_dict), pkl_file)
pkl_file.close()